# Top Tagging with Energy Flow Network

Documentation: [Examples](https://energyflow.network/examples/), [Architectures](https://energyflow.network/docs/archs/)

In [46]:
# Make tensorflow quieter
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Computing imports
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve

# Useful imports
from tqdm import tqdm
import matplotlib.pyplot as plt

# Energyflow imports
import energyflow as ef
from energyflow.archs import EFN
from energyflow.utils import data_split, to_categorical

In [ ]:
# Test tqdm, make sure it doesn't misbehave
for i in tqdm(range(10000000)):
    x = 1

In [23]:
# Make sure tensorflow can use the GPU
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# matplotlib settings
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.autolayout'] = True

## Load data

In [48]:
jets_path = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng/phys427/top-tagging/data/jets-100K-padded-shuffled.npz"
jets = np.load(jets_path)

X, y = jets["X"], jets["y"]
Y = y

print("Loaded data")

Loaded data


### Feature information

`X`: `(N, max_jet_size, 5)` 3D numpy array of floats
* `0`: $p_T$, transverse momentum
* `1`: $\eta$, angular coordinate
* `2`: $\phi$, angular coordinate
* `3`: $E$, energy
* `4`: $R$, $\sqrt{\eta^2 + \phi^2}$

`y`: `(N, 5)` 1D numpy array of ints that are one-hot encoded
* `0`: Gluon
* `1`: Light quark
* `2`: W boson
* `3`: Z boson
* `4`: Top quark


In [91]:
# Do train/val/test split
n_train = 75000
n_val = 10000
n_test = 15000

(z_train, z_val, z_test,
 p_train, p_val, p_test,
 Y_train, Y_val, Y_test) = data_split(X[:,:,0], X[:,:,1:3], Y, val=n_val, test=n_test)

print("Done train/val/test split")

Done train/val/test split


## Build the model

In [92]:
# Hyperparameters

# Network architecture parameters
Phi_sizes = [100, 100, 256]
F_sizes = [100, 100, 100]

# Network training parameters
num_epoch = 10
batch_size = 500

In [93]:
# Might take a few seconds
efn = EFN(input_dim=2,
          output_dim=5,
          Phi_sizes=Phi_sizes,
          F_sizes=F_sizes)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 phats_input (InputLayer)       [(None, None, 2)]    0           []                               
                                                                                                  
 tdist_0 (TimeDistributed)      (None, None, 100)    300         ['phats_input[0][0]']            
                                                                                                  
 activation_63 (Activation)     (None, None, 100)    0           ['tdist_0[0][0]']                
                                                                                                  
 tdist_1 (TimeDistributed)      (None, None, 100)    10100       ['activation_63[0][0]']          
                                                                                           

In [94]:
# Train the model!
efn.fit([z_train, p_train], Y_train,
        epochs=num_epoch,
        batch_size=batch_size,
        validation_data=([z_val, p_val], Y_val),
        verbose=1)

Epoch 1/10
150/150 [==============================] - 8s 20ms/step - loss: 1.1038 - acc: 0.5705 - val_loss: 0.9406 - val_acc: 0.6577
Epoch 2/10
150/150 [==============================] - 3s 17ms/step - loss: 0.8865 - acc: 0.6838 - val_loss: 0.8580 - val_acc: 0.6918
Epoch 3/10
150/150 [==============================] - 3s 17ms/step - loss: 0.8403 - acc: 0.6955 - val_loss: 0.8042 - val_acc: 0.7132
Epoch 4/10
150/150 [==============================] - 3s 17ms/step - loss: 0.8039 - acc: 0.7103 - val_loss: 0.7828 - val_acc: 0.7179
Epoch 5/10
150/150 [==============================] - 3s 17ms/step - loss: 0.7894 - acc: 0.7146 - val_loss: 0.7826 - val_acc: 0.7195
Epoch 6/10
150/150 [==============================] - 3s 17ms/step - loss: 0.7721 - acc: 0.7219 - val_loss: 0.7608 - val_acc: 0.7246
Epoch 7/10
150/150 [==============================] - 3s 17ms/step - loss: 0.7626 - acc: 0.7255 - val_loss: 0.7586 - val_acc: 0.7256
Epoch 8/10
150/150 [==============================] - 3s 17ms/step - 

## Evaluate model

In [95]:
# get predictions on test data
preds = efn.predict([z_test, p_test], batch_size=500)

30/30 [==============================] - 0s 6ms/step


In [98]:
test_labels = np.argmax(Y_test, axis=1)
pred_labels = np.argmax(preds, axis=1)

In [99]:
mask = (test_labels == pred_labels).astype(float)
print(mask)
print(f"Test accuracy: {mask.mean()}")

[1. 1. 0. ... 1. 1. 1.]
Test accuracy: 0.7278
